## 云车金融款型匹配

In [1]:
# coding=utf-8
import os
import re
import sys
import json
import math
import random
import datetime
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

ENCODING = 'utf-8'

PRODUCE_DB_ADDR_OUTTER = '172.17.0.4'
PRODUCE_DB_USER = 'root'
PRODUCE_DB_PASSWD = '123456'

LCCAL_ENGINE = 'mysql+pymysql://'+PRODUCE_DB_USER+':'+PRODUCE_DB_PASSWD+'@'+PRODUCE_DB_ADDR_OUTTER+'/temp?charset=utf8'
engine = create_engine(LCCAL_ENGINE)

query_sql = 'select * from base_model'
yunche_details = pd.read_sql_query(query_sql, engine)
yunche_details.to_csv('./yunche_details.csv', index=False)

query_sql = 'select * from base_series'
yunche_series = pd.read_sql_query(query_sql, engine)
yunche_series.to_csv('./yunche_series.csv', index=False)

query_sql = 'select * from base_brand'
yunche_brand = pd.read_sql_query(query_sql, engine)
yunche_brand.to_csv('./yunche_brand.csv', index=False)

print('finish query, num:'+str(len(yunche_details)))

finish query, num:30562


In [2]:
import sys
sys.path.append('../')

from match.match.match import Match

Using TensorFlow backend.


In [3]:
print('start project!')
result = pd.DataFrame()
car_source = pd.read_csv('./yunche_details.csv', low_memory=False)

detail_name = car_source.loc[:, ['full_name']].sort_values(by=['full_name'])
detail_name = detail_name.drop_duplicates(['full_name']).reset_index(drop=True)

print('等待匹配车源数据量:', len(car_source), '款型数量:', len(detail_name))
match = Match()
for i, detail in enumerate(list(detail_name.loc[:, 'full_name'].values)):

    part1 = car_source.loc[(car_source['full_name'] == detail), :].reset_index(drop=True)
    print(i, detail_name['full_name'][i], len(part1))
    gpj_detail = match.predict(part1['full_name'][0], cos_similar=0)['data']
    if len(gpj_detail) == 0:
        part1['origin_name'] = part1['full_name']
        part1 = part1.drop(['full_name'], axis=1)
        result = result.append(part1, sort=False).reset_index(drop=True)
    else:
        part2 = pd.DataFrame(gpj_detail, index=[0])
        part1['origin_name'] = part1['full_name']
        part1 = part1.drop(['full_name'], axis=1)
        temp = pd.concat([part1, pd.DataFrame(columns=list(part2.columns))], sort=False)
        temp.loc[:, part2.columns] = part2.loc[0, :].values
        result = result.append(temp, sort=False).reset_index(drop=True)

result.to_csv('./yunche_match_result.csv', index=False)
miss_match = result.loc[(result['brand_slug'].isnull()),:].reset_index(drop=True)
miss_match.to_csv('./miss_match.csv', index=False)

start project!
等待匹配车源数据量: 30562 款型数量: 30523
0 AC Schnitzer ACS3 2015款 3.0T 双离合 ACS3 sport 1
1 AC Schnitzer ACS5 2010款 3.0T 自动ACS35 猎鹰版 1
no acschnitzer [117] 2010 自动 3.0
no acschnitzer [117] 2010 自动 3.0
2 AC Schnitzer ACS5 2012款 3.0L 自动 35i Falcon 1
no acschnitzer [117] 2012 自动 3.0
no acschnitzer [117] 2012 自动 3.0
3 AC Schnitzer ACS5 2012款 3.0T 自动 40i 1
no acschnitzer [117] 2012 自动 3.0
no acschnitzer [117] 2012 自动 3.0
4 AC Schnitzer ACS5 2015款 3.0T 自动 ACS35 35i  1
5 ALPINA B4 Bi-Turbo 2017款 3.0T 自动 B4 Bi-Turbo Coupe 1
no alpina [276] 2017 自动 3.0
no alpina [276] 2017 自动 3.0
6 BRABUS巴博斯 GLE级 2016款 3.0T 自动 32GE 1
no 巴博斯 [140] 2016 自动 3.0
no 巴博斯 [140] 2016 自动 3.0
7 BRABUS巴博斯 GLE级 2016款 3.0T 自动 40GE 1
no 巴博斯 [140] 2016 自动 3.0
no 巴博斯 [140] 2016 自动 3.0
8 DS 3 2012款 1.6L 自动 时尚版 1
9 DS 3 2012款 1.6L 自动 至尊版 1
10 DS 3 2012款 1.6L 自动 风尚版 1
11 DS 3 2013款 1.6L 自动 时尚敞篷版 1
12 DS 3 2013款 1.6L 自动 至尊敞篷版 1
13 DS 3 2013款 1.6L 自动 风尚敞篷版 1
14 DS 4 2012款 1.6T 自动 雅致版 1
15 DS 4 2012款 1.6T 自动 风尚版 1
16 DS 4S 2016款 1

191 GMC SAVANA 2016款 6.0L 自动 G550L两驱雅尊版（对开门） 1
192 GMC SAVANA 2016款 6.0L 自动 G660L雅尊天幕版 1
193 GMC SAVANA 2016款 6.0L 自动 G660两驱雅尊版 1
194 GMC SAVANA 2017款 5.3L 自动 1500S 两驱经典版 1
195 GMC SAVANA 2017款 5.3L 自动 1500S 四驱经典版 1
196 GMC SAVANA 2017款 6.0L 自动 3500 公务版 1
197 GMC SAVANA 2017款 6.0L 自动 3500 行政版 1
198 GMC SAVANA 2017款 6.0L 自动 G550S 雅尊版 1
199 GMC SAVANA 2017款 6.0L 自动 G660 两驱雅尊版 1
200 GMC SAVANA 2017款 6.0L 自动 G660S 雅尊天幕版 1
201 GMC SAVANA 2017款 6.0L 自动 G760S 雅尊天玺型 1
202 GMC SAVANA 2017款 6.0L 自动 SAVANA 2500S至尊版 1
203 GMC SAVANA 2017款 6.0L 自动 SAVANA 3500 公务版 1
204 GMC SAVANA 2017款 6.0L 自动 SAVANA 3500 行政版 1
205 GMC SAVANA 2018款 6.0L 手自一体 尊翔星际版 1
206 GMC TERRAIN 2012款 3.0L 自动 标准型 1
207 GMC YUKON 2015款 5.3L 自动 2WD 1
208 GMC YUKON 2015款 5.3L 自动 XL SLE 4WD 1
209 GMC YUKON 2015款 5.3L 自动 XL SLT 4WD 1
210 GMC YUKON 2015款 6.2L 自动 DENALI 1
211 GMC YUKON 2015款 6.2L 自动 XL DENALI 4WD 1
212 GMC YUKON 2016款 5.3L 自动 XL豪华尊享加长版 1
213 GMC YUKON 2016款 5.3L 自动 豪华尊享版 1
214 GMC YUKON 2016款 6.2L 自动 DENALI至尊版 1
215 GM

403 Jeep 牧马人 2009款 3.8L 自动 四门版Rubicon 1
404 Jeep 牧马人 2010款 3.8L 自动 两门版Rubicon 1
405 Jeep 牧马人 2010款 3.8L 自动 两门版Sahara 1
406 Jeep 牧马人 2010款 3.8L 自动 四门版Rubicon 1
407 Jeep 牧马人 2010款 3.8L 自动 四门版Sahara 1
408 Jeep 牧马人 2011款 3.8L 自动 两门版Rubicon 1
409 Jeep 牧马人 2011款 3.8L 自动 两门版Sahara 1
410 Jeep 牧马人 2011款 3.8L 自动 四门版Rubicon 1
411 Jeep 牧马人 2011款 3.8L 自动 四门版Sahara 1
412 Jeep 牧马人 2012款 3.6L 自动 两门极地版Sahara 1
413 Jeep 牧马人 2012款 3.6L 自动 两门版Rubicon 1
414 Jeep 牧马人 2012款 3.6L 自动 两门版Sahara 1
415 Jeep 牧马人 2012款 3.6L 自动 四门梦十珍藏版 1
416 Jeep 牧马人 2012款 3.6L 自动 四门版Rubicon 1
417 Jeep 牧马人 2012款 3.6L 自动 四门版Sahara 1
418 Jeep 牧马人 2013款 3.6L 自动 两门Rubicon十周年限量版 1
419 Jeep 牧马人 2013款 3.6L 自动 两门版Rubicon 1
420 Jeep 牧马人 2013款 3.6L 自动 两门版Sahara 1
421 Jeep 牧马人 2013款 3.6L 自动 四门Rubicon十周年限量版 1
422 Jeep 牧马人 2013款 3.6L 自动 四门摩崖特别版 1
423 Jeep 牧马人 2013款 3.6L 自动 四门版Rubicon 1
424 Jeep 牧马人 2013款 3.6L 自动 四门版Sahara 1
425 Jeep 牧马人 2014款 2.8T 自动 四门Sahara柴油 1
426 Jeep 牧马人 2014款 3.0L 自动 四门Sahara 1
427 Jeep 牧马人 2014款 3.6L 自动 龙腾典藏版 1
428 Jeep 牧

621 MINI 2018款 2.0T 自动 COOPER S CABRIO 纪念版 1
622 MINI CABRIO 2004款 1.6L 自动 Cooper Cabrio 1
623 MINI CABRIO 2009款 1.6L 自动 Cooper Cabrio 1
624 MINI CABRIO 2009款 1.6T 自动 Cooper S Cabrio 1
625 MINI CABRIO 2011款 1.6L 自动 Cooper Cabrio 1
626 MINI CABRIO 2011款 1.6T 自动 Cooper S Cabrio 1
627 MINI CABRIO 2016款 1.5T 自动 COOPER CABRIO 1
628 MINI CABRIO 2016款 1.5T 自动 COOPER CABRIO 加勒比蓝限量版 1
629 MINI CABRIO 2016款 2.0T 自动 COOPER S CABRIO 1
630 MINI CABRIO 2016款 2.0T 自动 COOPER S CABRIO 加勒比蓝限量版 1
631 MINI CABRIO 2017款 1.5T 自动 COOPER CABRIO 自由派 1
632 MINI CABRIO 2017款 2.0T 自动 COOPER S CABRIO 自由派 1
633 MINI CABRIO 2017款 2.0T 自动 COOPER S CABRIO 赛车控 1
634 MINI CABRIO 2018款 1.5T 双离合 COOPER CABRIO 经典派 1
635 MINI CABRIO 2018款 1.5T 双离合 COOPER CABRIO 艺术家 1
636 MINI CABRIO 2018款 2.0T 自动 COOPER S CABRIO 纪念版 1
637 MINI CLUBMAN 2008款 1.6L 自动 Cooper Excitement 1
638 MINI CLUBMAN 2008款 1.6L 自动 Cooper Fun 1
639 MINI CLUBMAN 2008款 1.6T 自动 Cooper S 1
640 MINI CLUBMAN 2010款 1.6L 自动 Cooper Cheer 1
641 MINI CLUBMAN 2011款 1.6

804 Spirra 2011款 2.7L 手动 Spirra N 1
805 Spirra 2011款 2.7T 手动 Spirra EX 1
806 Spirra 2011款 2.7T 手动 Spirra S 1
807 Spirra 2011款 2.7T 手动 Spirra Turbo 1
808 WEY P8 2018款 2.0T 双离合 尊享版 1
809 WEY P8 2018款 2.0T 双离合 尊贵版 1
810 WEY P8 2018款 2.0T 双离合 旗舰版 1
811 WEY VV5 2017款 2.0T 双离合 旗舰型 1
812 WEY VV5 2017款 2.0T 双离合 超豪型 1
813 WEY VV5 2019款 2.0T 双离合 两驱 旗舰版 1
814 WEY VV5 2019款 2.0T 双离合 两驱 柚木棕旗舰版 1
815 WEY VV5 2019款 2.0T 双离合 两驱 超豪版 1
816 WEY VV5 2019款 2.0T 双离合 四驱 旗舰版 1
817 WEY VV6 2018款 2.0T 双离合 两驱旗舰版 1
818 WEY VV6 2018款 2.0T 双离合 两驱豪华版 1
819 WEY VV6 2018款 2.0T 双离合 两驱超豪版 1
820 WEY VV6 2018款 2.0T 双离合 四驱旗舰版 1
821 WEY VV6 2018款 2.0T 双离合 四驱超豪版 1
822 WEY VV7 2017款 2.0T 双离合 VV7c 旗舰型 1
823 WEY VV7 2017款 2.0T 双离合 VV7c 豪华型 1
824 WEY VV7 2017款 2.0T 双离合 VV7c 超豪型 1
825 WEY VV7 2017款 2.0T 双离合 VV7s 旗舰型 1
826 WEY VV7 2017款 2.0T 双离合 VV7s 豪华型 1
827 WEY VV7 2017款 2.0T 双离合 VV7s 超豪型 1
828 smart forfour 2016款 0.9T 双离合 BRABUS Xclusive 1
829 smart forfour 2016款 0.9T 双离合 先锋版 1
830 smart forfour 2016款 0.9T 双离合 吴亦凡特别版 1
831 sma

...

29678 雷克萨斯 IS 2007款 3.0L 自动 300炫动特别版 1
29679 雷克萨斯 IS 2008款 3.0L 自动 300豪华版 1
29680 雷克萨斯 IS 2009款 3.0L 自动 300 1
29681 雷克萨斯 IS 2009款 3.0L 自动 300C 1
29682 雷克萨斯 IS 2009款 3.0L 自动 300炫动版 1
29683 雷克萨斯 IS 2009款 3.0L 自动 300豪华版 1
29684 雷克萨斯 IS 2011款 2.5L 自动 250 F-Sport 1
29685 雷克萨斯 IS 2011款 2.5L 自动 250C 1
29686 雷克萨斯 IS 2011款 2.5L 自动 250炫动版 1
29687 雷克萨斯 IS 2011款 2.5L 自动 250精英版 1
29688 雷克萨斯 IS 2011款 2.5L 自动 250豪华版 1
29689 雷克萨斯 IS 2011款 2.5L 自动 250运动版 1
29690 雷克萨斯 IS 2011款 2.5L 自动 250风驰版 1
29691 雷克萨斯 IS 2013款 2.5L 自动 250 F-Sport 1
29692 雷克萨斯 IS 2013款 2.5L 自动 250尊贵版 1
29693 雷克萨斯 IS 2013款 2.5L 自动 250豪华版 1
29694 雷克萨斯 IS 2013款 2.5L 自动 250锋尚版 1
29695 雷克萨斯 IS 2013款 2.5L 自动 250领先版 1
29696 雷克萨斯 IS 2015款 2.0T 自动 200t F SPORT 1
29697 雷克萨斯 IS 2015款 2.0T 自动 200t 豪华版 1
29698 雷克萨斯 IS 2015款 2.0T 自动 200t 锋尚版 1
29699 雷克萨斯 IS 2015款 2.0T 自动 200t 领先版 1
29700 雷克萨斯 IS 2015款 2.5L 自动 逐风之翼限量版 1
29701 雷克萨斯 IS 2017款 2.0T 自动 300 F SPORT版 1
29702 雷克萨斯 IS 2017款 2.0T 自动 300 豪华版 1
29703 雷克萨斯 IS 2017款 2.0T 自动 300 锋尚版 1
29704 雷克萨斯 I

29883 雷诺 科雷傲 2010款 2.5L CVT 两驱舒适版 1
29884 雷诺 科雷傲 2010款 2.5L CVT 两驱豪华版 1
29885 雷诺 科雷傲 2010款 2.5L CVT 四驱BOSE限量版 1
29886 雷诺 科雷傲 2010款 2.5L CVT 四驱舒适版 1
29887 雷诺 科雷傲 2010款 2.5L CVT 四驱豪华版 1
29888 雷诺 科雷傲 2012款 2.5L CVT 两驱舒适导航版 1
29889 雷诺 科雷傲 2012款 2.5L CVT 两驱豪华导航版 1
29890 雷诺 科雷傲 2012款 2.5L CVT 两驱都市版 1
29891 雷诺 科雷傲 2012款 2.5L CVT 四驱舒适导航版 1
29892 雷诺 科雷傲 2012款 2.5L CVT 四驱豪华导航版 1
29893 雷诺 科雷傲 2012款 2.5L CVT 四驱都市版 1
29894 雷诺 科雷傲 2013款 2.5L CVT 两驱舒适导航版 1
29895 雷诺 科雷傲 2013款 2.5L CVT 两驱舒适版 1
29896 雷诺 科雷傲 2013款 2.5L CVT 两驱豪华导航版 1
29897 雷诺 科雷傲 2013款 2.5L CVT 两驱豪华版 1
29898 雷诺 科雷傲 2013款 2.5L CVT 两驱都市版 1
29899 雷诺 科雷傲 2013款 2.5L CVT 四驱舒适导航版 1
29900 雷诺 科雷傲 2013款 2.5L CVT 四驱舒适版 1
29901 雷诺 科雷傲 2013款 2.5L CVT 四驱豪华导航版 1
29902 雷诺 科雷傲 2013款 2.5L CVT 四驱豪华版 1
29903 雷诺 科雷傲 2013款 2.5L CVT 四驱都市版 1
29904 雷诺 科雷傲 2014款 2.0L CVT 两驱时尚型岁末限量导航版 1
29905 雷诺 科雷傲 2014款 2.0L CVT 两驱时尚版 1
29906 雷诺 科雷傲 2014款 2.0L CVT 两驱时尚版(改款) 1
29907 雷诺 科雷傲 2014款 2.0L CVT 两驱智享版 1
29908 雷诺 科雷傲 2014款 2.0L CVT 两驱智享版(改款) 1
29909 雷诺 科雷傲 2014款 2.0L CVT 两

30102 马自达 Mazda2三厢 2010款 1.3L 自动 劲翔 炫动款标准版 1
no 马自达 [58] 2010 自动 1.3
no 马自达 [58] 2010 自动 1.3
30103 马自达 Mazda2三厢 2010款 1.5L 手动 劲翔 炫动款时尚版 1
no 马自达 [58] 2010 手动 1.5
no 马自达 [58] 2010 手动 1.5
30104 马自达 Mazda2三厢 2010款 1.5L 自动 劲翔 炫动款时尚版 1
no 马自达 [58] 2010 自动 1.5
no 马自达 [58] 2010 自动 1.5
30105 马自达 Mazda2三厢 2010款 1.5L 自动 劲翔 炫动款豪华版 1
no 马自达 [58] 2010 自动 1.5
no 马自达 [58] 2010 自动 1.5
30106 马自达 Mazda2三厢 2011款 1.3L 手动 劲翔 炫动款标准型 1
30107 马自达 Mazda2三厢 2011款 1.3L 自动 劲翔 炫动款标准型 1
30108 马自达 Mazda2三厢 2011款 1.5L 手动 劲翔 炫动款时尚型 1
30109 马自达 Mazda2三厢 2011款 1.5L 自动 劲翔 炫动款时尚型 1
30110 马自达 Mazda2三厢 2011款 1.5L 自动 劲翔 炫动款豪华型 1
30111 马自达 Mazda2三厢 2012款 1.3L 手动 三厢 炫动款标准版 1
30112 马自达 Mazda2三厢 2012款 1.3L 自动 三厢 炫动款标准版 1
30113 马自达 Mazda2三厢 2012款 1.5L 手动 三厢 炫动款时尚版 1
30114 马自达 Mazda2三厢 2012款 1.5L 自动 三厢 炫动款豪华版 1
30115 马自达 Mazda2三厢 2012款 1.5L 自动 炫动款时尚版 1
30116 马自达 Mazda2两厢 2007款 1.3L 手动 两厢标准版 1
30117 马自达 Mazda2两厢 2007款 1.3L 自动 两厢时尚版 1
30118 马自达 Mazda2两厢 2007款 1.3L 自动 两厢标准版 1
30119 马自达 Mazda2两厢 2008款 1.3L 手动 两厢标准版 1
30120 马自达 Mazda2两

30294 马自达 昂克赛拉两厢 2017款 1.5L 自动 两厢舒适型 1
30295 马自达 昂克赛拉两厢 2017款 1.5L 自动 两厢豪华型 1
30296 马自达 昂克赛拉两厢 2017款 2.0L 自动 两厢豪华型 1
30297 马自达 昂克赛拉两厢 2017款 2.0L 自动 两厢运动型 1
30298 马自达 睿翼 2008款 2.0L 手动 1
30299 马自达 睿翼 2009款 2.0L 自动 1
30300 马自达 睿翼 2009款 2.5L 自动 导航版 1
30301 马自达 睿翼 2009款 2.5L 自动 尊贵版 1
30302 马自达 睿翼 2009款 2.5L 自动 至尊版 1
30303 马自达 睿翼 2010款 2.0L 手动 1
30304 马自达 睿翼 2010款 2.0L 自动 豪华版 1
30305 马自达 睿翼 2011款 2.0L 自动 精英版 1
30306 马自达 睿翼 2012款 2.0L 自动 精英版 1
30307 马自达 睿翼 2012款 2.0L 自动 豪华版 1
30308 马自达 睿翼 2012款 2.5L 自动 尊贵版 1
30309 马自达 睿翼 2012款 2.5L 自动 至尊版 1
30310 马自达 睿翼 2015款 2.0L 自动 精英版 1
30311 马自达 睿翼 2015款 2.0L 自动 豪华版 1
30312 马自达 睿翼轿跑 2010款 2.0L 自动 轿跑豪华版 1
30313 马自达 睿翼轿跑 2010款 2.5L 自动 轿跑至尊版 1
30314 马自达 睿翼轿跑 2011款 2.0L 自动 轿跑精英版 1
30315 马自达 睿翼轿跑 2012款 2.0L 自动 轿跑精英版 1
30316 马自达 睿翼轿跑 2012款 2.0L 自动 轿跑豪华版 1
30317 马自达 睿翼轿跑 2012款 2.5L 自动 轿跑至尊版 1
30318 马自达 睿翼轿跑 2015款 2.0L 自动 轿跑精英版 1
30319 马自达 睿翼轿跑 2015款 2.0L 自动 轿跑豪华版 1
30320 马自达 阿特兹 2014款 2.0L 自动 蓝天尊贵版 1
30321 马自达 阿特兹 2014款 2.0L 自动 蓝天时尚版 1
30322 马自达 阿特兹 2014款 2.0L 自

30494 黄海 旗胜F1 2011款 2.4L 手动 四驱超豪华型汽油版 1
30495 黄海 旗胜F1 2011款 2.4L 自动 两驱超豪华型汽油版 1
no 黄海 [97] 2011 自动 2.4
no 黄海 [97] 2011 自动 2.4
30496 黄海 旗胜F1 2011款 2.5T 手动 两驱豪华型柴油版 1
no 黄海 [97] 2011 手动 2.5
no 黄海 [97] 2011 手动 2.5
30497 黄海 旗胜F1 2012款 2.0L 手动 两驱公务版 1
30498 黄海 旗胜F1 2012款 2.0L 手动 两驱至尊版 1
30499 黄海 旗胜F1 2012款 2.0L 手动 两驱运动版 1
30500 黄海 旗胜V3 2011款 2.0L 手动 精英版 1
30501 黄海 旗胜V3 2011款 2.0L 手动 超豪华型 1
30502 黄海 旗胜V3 2011款 2.4L 手动 豪华型 1
30503 黄海 旗胜V3 2011款 2.4L 手动 超豪华型 1
30504 黄海 旗胜V3 2013款 1.9T 手动 柴油版DD6472C 1
no 黄海 [97] 2013 手动 1.9
no 黄海 [97] 2013 手动 1.9
30505 黄海 翱龙CUV 2006款 2.4L 手动 DD6480 4×2 JX493ZQ3豪华型 1
no 黄海 [97] 2006 手动 2.4
no 黄海 [97] 2006 手动 2.4
30506 黄海 翱龙CUV 2006款 2.4T 手动 DD6480A 4×4 4G64S4M豪华型 1
no 黄海 [97] 2006 手动 2.4
no 黄海 [97] 2006 手动 2.4
30507 黄海 翱龙CUV 2006款 2.8L 手动 舒适型后驱柴油 1
no 黄海 [97] 2006 手动 2.8
no 黄海 [97] 2006 手动 2.8
30508 黄海 翱龙CUV 2009款 2.0L 手动 两驱标准型DD6470N 1
30509 黄海 翱龙CUV 2009款 2.0L 手动 两驱舒适型 DD6470 1
30510 黄海 翱龙CUV 2009款 2.0L 手动 两驱豪华型 DD6470 1
30511 黄海 翱龙CUV 2009款 2.4L 手动 四驱豪华型 DD64

In [3]:
yunche_brand = pd.read_csv('./yunche_brand.csv')
yunche_series = pd.read_csv('./yunche_series.csv')
yunche_details = pd.read_csv('./yunche_details.csv')
yunche_match_result = pd.read_csv('./yunche_match_result.csv')
combine_detail = pd.read_csv('../tmp/train/combine_detail.csv')
yunche_match_result = yunche_match_result.merge(combine_detail[['detail_model_slug','id']].rename(columns={'detail_model_slug':'gpj_detail_slug',
                                                                                                          'id':'model_detail_slug_id'}),how='left',on=['gpj_detail_slug'])

yunche_details = yunche_details.merge(yunche_brand[['code','name']].rename(columns={'code':'brand','name':'origin_brand_name'}),how='left',on=['brand'])
yunche_details = yunche_details.merge(yunche_series[['code','name']].rename(columns={'code':'series','name':'origin_model_name'}),how='left',on=['series'])
yunche_details = yunche_details[['code','origin_brand_name','origin_model_name','name','full_name','model_year']]
yunche_details = yunche_details.rename(columns={'code':'model_detail_origin_id','origin_brand_name':'brand','origin_model_name':'model',
                                               'name':'model_detail','full_name':'cleaned_detail','model_year':'year'})
yunche_details = yunche_details.merge(yunche_match_result[['code','model_name','gpj_model_slug','volume','model_detail_slug_id']].rename(columns={
    'code':'model_detail_origin_id','model_name':'global_name','gpj_model_slug':'global_slug'}),how='left',on=['model_detail_origin_id'])
yunche_details['domain'] = 'yunche.com'
yunche_details['status'] = 'A'
yunche_details.loc[(yunche_details['model_detail_slug_id'].isnull()), 'status'] = 'P'
yunche_details.to_csv('./model_detail_normal_yunche.csv',index=False)
yunche_details.info()

/home/ml/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (5,9,11,22,24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 30562 entries, 0 to 30561
Data columns (total 12 columns):
model_detail_origin_id    30562 non-null int64
brand                     30562 non-null object
model                     30562 non-null object
model_detail              30562 non-null object
cleaned_detail            30562 non-null object
year                      30562 non-null int64
global_name               28569 non-null object
global_slug               28569 non-null object
volume                    28569 non-null float64
model_detail_slug_id      28549 non-null float64
domain                    30562 non-null object
status                    30562 non-null object
dtypes: float64(2), int64(2), object(8)
memory usage: 3.0+ MB


In [6]:
import pandas as pd
from sqlalchemy import create_engine

ENCODING = 'utf-8'

PRODUCE_DB_ADDR_OUTTER = '101.200.229.249'
PRODUCE_DB_USER = 'pingjia'
PRODUCE_DB_PASSWD = 'De32wsxC'

TEST_PINGJIA_ENGINE = 'mysql+pymysql://'+PRODUCE_DB_USER+':'+PRODUCE_DB_PASSWD+'@'+PRODUCE_DB_ADDR_OUTTER+'/pingjia?charset=utf8'
engine = create_engine(TEST_PINGJIA_ENGINE, encoding=ENCODING)

with engine.begin() as con:
    sql = 'delete from pingjia.model_detail_normal where domain = \'yunche.com\' '
    con.execute(sql)
con.close()
yunche_details.to_sql(name='model_detail_normal', if_exists='append', con=engine, index=False)

In [7]:
import pandas as pd
from sqlalchemy import create_engine

ENCODING = 'utf-8'

PRODUCE_DB_ADDR_OUTTER = '101.201.143.74'
PRODUCE_DB_USER = 'leidengjun'
PRODUCE_DB_PASSWD = 'ldj_DEV_~!'
PRODUCE_PINGJIA_ENGINE = 'mysql+pymysql://'+PRODUCE_DB_USER+':'+PRODUCE_DB_PASSWD+'@'+PRODUCE_DB_ADDR_OUTTER+'/pingjia?charset=utf8'
engine = create_engine(PRODUCE_PINGJIA_ENGINE, encoding=ENCODING)

sql = 'select name,city_administration_number from pingjia.open_city where parent != 0'
data =  pd.read_sql_query(sql, engine)

In [10]:
data.to_csv('./city.csv',index=False)

In [12]:
sql = 'select color from pingjia.open_color'
data =  pd.read_sql_query(sql, engine)
data.to_csv('./color.csv',index=False)

In [18]:
sql = 'http://openapi.gongpingjia.com/api/cars/evaluation/yunche/?d_model=8081&year=2018&month=3&color=白&mile=10&city=110100&condition_desc=[{"item":"左前纵梁","number":1},{"item":"右前纵梁","number":0},{"item":"左前减震器座","number":0},{"item":"右前减震器座","number":0},{"item":"防火墙","number":0},{"item":"左A柱","number":0},{"item":"左B柱","number":0},{"item":"左C柱","number":0},{"item":"右B柱","number":0},{"item":"右C柱","number":0},{"item":"右A柱","number":0},{"item":"左后纵梁","number":0},{"item":"右后纵梁","number":0},{"item":"右后减震器座","number":0},{"item":"左后减震器座","number":0},{"item":"火烧情况","number":0},{"item":"泡水情况","number":0},{"item":"前保险杠","number":0},{"item":"机盖","number":0},{"item":"左前叶子板","number":0},{"item":"左前轮胎","number":0},{"item":"左前轮毂","number":0},{"item":"前挡风玻璃","number":0},{"item":"左前门","number":0},{"item":"左后门","number":0},{"item":"左后叶子板","number":0},{"item":"左后轮胎","number":0},{"item":"左后轮毂","number":0},{"item":"左后大灯","number":0},{"item":"右后大灯","number":0},{"item":"后保险杠","number":0},{"item":"右后轮胎","number":0},{"item":"右后轮毂","number":0},{"item":"右后叶子板","number":0},{"item":"右后门","number":0},{"item":"右前门","number":0},{"item":"右前叶子板","number":0},{"item":"右前轮胎","number":0},{"item":"右前轮毂","number":0},{"item":"左侧下边梁","number":0},{"item":"右侧下边梁","number":0},{"item":"车顶","number":0},{"item":"右前大灯","number":0},{"item":"左前大灯","number":0},{"item":"中控台","number":0},{"item":"仪表盘","number":0},{"item":"主驾驶座椅","number":0},{"item":"副驾驶座椅","number":0},{"item":"后排座椅","number":0},{"item":"车辆顶棚","number":0},{"item":"换挡杆区域","number":0},{"item":"车辆娱乐设备","number":0},{"item":"方向盘","number":0},{"item":"空调","number":0},{"item":"右前翼子板内衬","number":0},{"item":"左前翼子板内衬","number":0},{"item":"右后翼子板内衬","number":0},{"item":"左后翼子板内衬","number":0},{"item":"前防撞梁","number":0},{"item":"后防撞梁","number":0},{"item":"备胎槽","number":0},{"item":"水箱框架","number":0},{"item":"发动机","number":0},{"item":"发动机状况","number":0},{"item":"变速箱","number":0},{"item":"变速箱状况","number":0},{"item":"转向系统","number":0},{"item":"离合器系统","number":0},{"item":"刹车系统","number":0},{"item":"发动机是否烧机油","number":0}]'
len(sql)

2040